# Support Vector Machines (SVM)

Another machine learning algorithm is the support vector machine (SVM). It provides resourceful predictions for complex real-word problems, reduces redundant information, and is ideal for smaller datasets. Some real-world examples of when SVMs can be used are bioinformatics, face recognition, speech recognition, or cancer detection.

To perform a classification using the support vector machine algorithm, complete the following:

1. Access the "UCI Machine Learning Repository," located in the topic Resources. Note: There are about 120 data sets that are suitable for use in a classification task. For this part of the exercise, you must choose one of these datasets, provided it includes at least 10 attributes and 10,000 instances.
2. Ensure that the datasets are suitable for classification using this method.
3. You may search for data in other repositories, such as Data.gov, Kaggle or Scikit Learn.
4. Examine the repository through which you accessed the dataset and discuss data management measures set in place, such as protecting the privacy of those accessing the site and protecting the intellectual property rights of the data owners/contributors.


For your selected dataset, build a classification model as follows:

1. Explain the dataset and the type of information you wish to gain by applying a classification method.
2. Explain what makes SVM algorithm very special and very different from most other machine learning algorithms. Explain how you will be using it in your analysis (list the steps, the intuition behind the mathematical representation, and address its assumptions).
3. Explain the concepts: kernel, hyperplane, and decision boundary, and their role in SVM.
4. Explain the concepts: maximum margin, support vectors, and maximum margin hyperplane, and their role in SVM.
5. Import the necessary libraries, then read the dataset into a data frame and perform initial statistical exploration.
6. Clean the data and address unusual phenomena (e.g., normalization, feature scaling, outliers); use illustrative diagrams and plots and explain them.
7. Formulate two questions that can be answered by applying a classification method using the SVM
8. Split the data into 80% training and 20% testing sets using the train test split class.
9. Use a linear kernel to train the SVM classifier on the training set (e.g., fit the support vector regressor to the dataset). Explain the intuition behind each of the key mathematical steps.
10. Explain the choice of the optimal hyperplane.
11. Make classification predictions.
12. Interpret the results in the context of the questions you asked.
13. Validate your model using a confusion matrix, accuracy score, ROC-AUC curves, and k-fold cross validation. Then, explain the results.
14. Include all mathematical formulas used and graphs representing the final outcomes.


Prepare a comprehensive technical report as a markdown document or Jupyter notebook, including all code, code comments, all outputs, plots, and analysis. Make sure the project documentation contains a) problem statement, b) algorithm of the solution, c) analysis of the findings, and d) references.

### Problem Statement

- Contains Patient Reviews on specific drugs with related conditions.
- Contains a 10-star rating system to measure overall patient satisfaction.
- Useful for sentiment analysis of drug experience over multiple aspects of the drug
- Contains data from multiple sources (may be about bringing confidence about new drugs to patients).

In [2]:
import pandas as pd

df__drug_review__train = pd.read_csv('drugsComTrain_raw.tsv', sep='\t')
df__drug_review__test = pd.read_csv('drugsComTest_raw.tsv', sep='\t')
df__drug_review = pd.concat([df__drug_review__test,df__drug_review__train])